# Intelligenza Artificiale - Lab 3

In questo laboratorio esploreremo un problema di classificazione realistico che coinvolge dati ad alta dimensionalità, ovvero, la classificazione di immagini. Le simulazioni si baseranno su un dataset molto popolare chiamato [MNIST](https://en.wikipedia.org/wiki/MNIST_database), che contiene 70.000 immagini di cifre manoscritte con le relative etichette (da 0 a 9).

Per risolvere questo problema implementeremo una semplice architettura di ***deep learning***, ovvero un MLP dotato di vari strati nascosti. Vedremo anche come implementare una variante **convoluzionale**, particolarmente adatta al processamento di immagini.

Vedremo infine come simulare un compito percettivo, ricavando una curva psicometrica che descrive la sensibilità del modello rispetto al rumore contenuto negli stimoli visivi.

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from torchvision.datasets import MNIST
from torchvision.transforms import Lambda

In [ ]:
%%capture
mnist_train = MNIST(root="../mnist",
                    train=True,      # dati di training
                    download=True)
mnist_test = MNIST(root="../mnist",
                    train=False,     # dati di test
                    download=True)

In [ ]:
mnist_tr_in, mnist_tr_out = mnist_train.data.numpy(), mnist_train.targets.numpy()
mnist_te_in, mnist_te_out = mnist_test.data.numpy(), mnist_test.targets.numpy()

Le immagini sono salvate in formato bi-dimensionale (matrici 28x28). Tuttavia la rete neurale "fully connected" riceve in input vettori uni-dimensionali, quindi come primo step "linearizziamo" le matrici con l'operazione di `reshape`, ottenendo vettori di 784 elementi.

Inoltre le immagini sono salvate in un formato convenzionale per le immagini, dove ciascun pixel può assumere valori tra 0 e 255. Come secondo step normalizziamo quindi i valori nell'intervallo tra 0 e 1, semplicemente dividendo per 255.

In [ ]:
mnist_tr_in = mnist_tr_in.reshape(60000, 28*28)
mnist_te_in = mnist_te_in.reshape(10000, 28*28)

In [ ]:
mnist_tr_in = mnist_tr_in / 255
mnist_te_in = mnist_te_in / 255

Creiamo ora un MLP con due strati nascosti, lasciando i parametri di apprendimento di default, e procediamo con l'apprendimento.

In [ ]:
# con 10 iterazioni la convergenza è già abbastanza buona e l'algoritmo
# impiega circa 3 minuti a completare l'esecuzione. Volendo una convergenza
# migliore si dovrebbe aumentare il numero di iterazioni (e di hidden layers)

random_state = 0

MLP = MLPClassifier(hidden_layer_sizes=(500, 500),
                    max_iter = 10,
                    random_state=random_state)

In [ ]:
MLP = MLP.fit(mnist_tr_in, mnist_tr_out)

Possiamo ora procedere visualizzando la curva dell'errore, l'accuratezza media e la matrice di confusione.

In [ ]:
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

In [ ]:
_ = plt.plot(range(MLP.n_iter_), MLP.loss_curve_)
_ = plt.xlabel("Epoca")
_ = plt.ylabel("Loss")
_ = plt.title("Loss durante l'apprendimento")
plt.ylim(0, 0.3);

In [ ]:
MLP.score(mnist_te_in, mnist_te_out)

In [ ]:
test_predictions = MLP.predict(mnist_te_in)

In [ ]:
_ = metrics.ConfusionMatrixDisplay.from_predictions(mnist_te_out, test_predictions)

### Curve psicometriche: resistenza al rumore

Creiamo una funzione che prende in input una matrice di dati (nel nostro caso, immagini MNIST) ed un livello di rumore desiderato e restituisce una versione rumorosa delle immagini.

In [ ]:
def _inject_Gaussian_noise(mnist_data, noise_level):
  # creiamo una matrice di rumore della dimensione dell'intero dataset dato in input alla funzione
  dataset_size = mnist_data.shape
  random_gaussian_vector = np.random.normal(loc = 0, scale = noise_level, size = dataset_size)
  # aggiungiamo il rumore ai pixel originali, tagliando i valori minori di 0 o maggiori di 1
  noisy_images = mnist_data + random_gaussian_vector
  noisy_images = np.clip(noisy_images,0,1)
  return noisy_images

In [ ]:
noise_level = 0.3  # varianza della Gaussiana
mnist_te_with_noise = _inject_Gaussian_noise(mnist_te_in, noise_level)

In [ ]:
_ = plt.imshow(mnist_te_with_noise[0].reshape(28, 28), cmap="gray")

Possiamo richiamare la funzione appena creata per più volte, incrementando di volta in volta il livello di rumore. In questo modo possiamo vedere come il numero di errori cresce all'aumentare del rumore.

In [ ]:
livelli_di_rumore = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
percentuale_errori = []

# testiamo il modello su stimoli contenenti un livello di rumore crescente:
for livello in livelli_di_rumore:
  mnist_con_rumore = _inject_Gaussian_noise(mnist_te_in, livello)
  accuratezza = MLP.score(mnist_con_rumore, mnist_te_out)
  percentuale_errori.append(1-accuratezza)


In [ ]:
_ = plt.plot(livelli_di_rumore, percentuale_errori)
_ = plt.xlabel("Rumore")
_ = plt.ylabel("Percentuale di errori")
_ = plt.title("Curva psicometrica MLP")

## Convolutional Neural Network

Creiamo ora una semplice variante convoluzionale dell'architettura di deep learning. La rete pre-impostata ha un singolo layer convoluzionale (con 32 filtri) ed un layer fully-connected con 50 neuroni nascosti.

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.models.Sequential(
    [keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
     keras.layers.Flatten(),
     keras.layers.Dense(units=50, activation='softmax')]
)

In [ ]:
model.compile(optimizer='adam',
              metrics=["accuracy"],
              loss='sparse_categorical_crossentropy')

La CNN richiede dati di input 2D, a differenza di MLP che richiede vettori 1D. Dobbiamo quindi riportare i dati in formato immagine bi-dimensionale:

In [ ]:
mnist_tr_in_conv = mnist_tr_in.reshape(-1, 28, 28, 1)
mnist_te_in_conv = mnist_te_in.reshape(-1, 28, 28, 1)

In [ ]:
history = model.fit(mnist_tr_in_conv, mnist_tr_out, epochs=5)

In [ ]:
_ = plt.plot(range(5), history.history['loss'])
_ = plt.xlabel("Epoca")
_ = plt.ylabel("Loss")
_ = plt.title("Loss durante l'apprendimento")

In [ ]:
test_loss, test_accuracy = model.evaluate(mnist_te_in_conv, mnist_te_out)

In [ ]:
test_predictions_conv = model.predict(mnist_te_in_conv)

In [ ]:
_ = metrics.ConfusionMatrixDisplay.from_predictions(mnist_te_out,
                                                    test_predictions_conv.argmax(axis=1))

### Resistenza al rumore

In [ ]:
livelli_di_rumore = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
percentuale_errori_conv = []

# testiamo il modello su stimoli contenenti un livello di rumore crescente:
for livello in livelli_di_rumore:
  mnist_con_rumore = _inject_Gaussian_noise(mnist_te_in, livello)
  mnist_con_rumore_conv = mnist_con_rumore.reshape(-1, 28, 28, 1)
  _ , accuratezza = model.evaluate(mnist_con_rumore_conv, mnist_te_out)
  percentuale_errori_conv.append(1-accuratezza)


In [ ]:
_ = plt.plot(livelli_di_rumore, percentuale_errori, label = 'MLP')
_ = plt.plot(livelli_di_rumore, percentuale_errori_conv, label = 'CNN')
_ = plt.legend()
_ = plt.xlabel("Rumore")
_ = plt.ylabel("Errore")
_ = plt.title("Resistenza al rumore MLP vs. CNN")